# Hands on: Persistent Homology

In this notebook, you will find some exercises to familiarize yourself with the basics of persistence diagrams and the Ripser library.  
(See more at https://github.com/ctralie/TDALabs)

In [ ]:
# Perform all of the necessary library imports.  
import numpy as np
import matplotlib.pyplot as plt
from ripser import Rips
import time
from fermat import Fermat
from scipy.spatial import distance_matrix

## **Example 1:** Circles

As a first example, we will make a circle.  Once you understand this example, starting from the code below, see if you can modify the point cloud in order to:

#### **Exercises/Questions**
* Increase the birth time of the $H_1$ generator without changing the radius of the circle
* Decrease the birth time of the $H_1$ generator without changing the radius of the circle
* What happens to $H_1$ when you add a point that is at the center of the circle at position $(0, 0)$?
* Change the point cloud so that there are two $H_1$ generators with large (>1) but different persistences.

In [ ]:
# Create 100 points on the unit circle
N = 100
t = np.linspace(0, 2*np.pi, N+1)[0:N]
X = np.zeros((N, 2))
X[:, 0] = np.cos(t)
X[:, 1] = np.sin(t)


# Compute the persistence diagram of this point cloud
rips = Rips()
dgms = rips.fit_transform(X)

# Plot the figures
plt.figure(figsize=(8, 4))

plt.subplot(121)
plt.scatter(X[:, 0], X[:, 1], alpha = 0.5)
plt.axis('equal')
plt.title("Point Cloud")

plt.subplot(122)
rips.plot(dgms)
plt.title("Persistence Diagram")

plt.show()

## **Example 2:** Sphere
We will now randomly sample some points on the two dimensional sphere and compute persistent homology up to dimension 2, using the <i>geodesic distance</i>, which is the arc length between points.  A point $X = (x, y, z)$ is randomly drawn from a 3D gaussian distribution 

$$(x, y, z) \sim \left(\frac{1}{2 \pi}\right)^{3/2} \exp{\left( - (x^2+y^2+z^2)/2 \right) }$$

and normalized to have a length of 1 

(<i>Why does this evenly sample on the sphere?</i> See Marsaglia, G. _"Choosing a Point from the Surface of a Sphere."_ Ann. Math. Stat. 43, 645-646, 1972.
or Muller, M. E. _"A Note on a Method for Generating Points Uniformly on N-Dimensional Spheres."_ Comm. Assoc. Comput. Mach. 2, 19-20, Apr. 1959).

Note that the arc length along a unit circle is equal to the angle subtending that circle in radians. Thus, given two unit vectors on the sphere $x_1$ and $x_2$, the arc length between them is $\cos^{-1}(x_1 \cdot x_2)$.  We will implement this in code below to compute all pairwwise arc lengths between our sampled points on the sphere.  We will then compute a Rips filtration up to dimension 2.


In [ ]:
# Step 1: Setup the point cloud
N = 200 #Number of points
X = np.random.randn(N, 3) #Draw from 3D Gaussian
X = X/np.sqrt(np.sum(X**2, 1))[:, None] #Normalize each point to unit length

# Step 2: Compute all pairwise arc lengths between sampled points
dotProds = X.dot(X.T) #Entry ij in this matrix holds the dot product between point i and j, or cos(theta)
#The dot products should be in [-1, 1], but may leave this range due to numerical roundoff
dotProds[dotProds < -1] = -1
dotProds[dotProds > 1] = 1
DSphere = np.arccos(dotProds) #The arc length is the inverse cosine of the dot products of unit vectors
np.fill_diagonal(DSphere, 0) #Be sure distance of points to themselves is zero

# Step 3: Compute the persistence diagram of this metric space, described by its distance matrix
rips = Rips(maxdim=2)
dgms = rips.fit_transform(DSphere, distance_matrix=True)

# Step 4: Plot the figures
fig = plt.figure(figsize=(10, 4))
ax = plt.subplot(121, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], alpha = 0.5)
plt.title("Sphere Sample")
plt.subplot(122)
rips.plot(dgms)
plt.title("Persistence Diagram");


#### **Questions**
* How many generators with long lifetime are in $H_1$ and $H_2$?  Does this match the homology of a sphere?
* Compare the persistence diagram of the sphere endowed with either the Euclidean distance and the geodesic distance. Is there any change?

## **Example 3:** 3D Embedded Torus

We will now sample points from a torus embedded in 3D and compute persistent homology up to $H_2$.  Given a parameter $R$ which is the radius of a circle through the center of the tube and a parameter $r$ which is the width of the tube, we use the following equation for the 3D coordinates of a point $(\theta, \phi)$, where $\theta$ is along the $r$ radius circle and $\phi$ is along the $R$ radius circle:

$x = (R + r \cos(\theta)) \cos (\phi)$

$y = (R + r \cos(\theta)) \sin (\phi)$

$z = r \sin (\theta)$

#### **Questions**

* What happens to the persistence diagram as you vary $R$ and $r$?
* What happens to the persistence diagram as you vary $n\_sample$?

In [ ]:
N = 10000 #Number of initial points in (theta, phi) space
n_sample = 400 #Number of points to evenly subsample in 3D

R = 4
r = 2
theta = np.random.rand(N)*2*np.pi
phi = np.random.rand(N)*2*np.pi

X = np.zeros((N, 3))
X[:, 0] = (R + r*np.cos(theta))*np.cos(phi)
X[:, 1] = (R + r*np.cos(theta))*np.sin(phi)
X[:, 2] = r*np.sin(theta)
xr = [np.min(X.flatten()), np.max(X.flatten())]

sample = np.random.choice(N, size=n_sample, replace=False)
X_sample = X[sample]

#Now compute persistence diagrams up to H2 on the subsample


#Plot the figures


## **Example 4:**  4D Embedded Klein Bottle

Though we can't embed the Klein bottle in 3 dimensions, we can in 4 dimensions with the following equations, which are similar to the 3D torus case but with a twist

$x = (R + r \cos(\theta)) \cos (\phi)$

$y = (R + r \cos(\theta)) \sin (\phi)$

$z = r \sin (\theta) \cos(\phi /2)$

$w = r \sin (\theta) \sin(\phi / 2)$

Let's now run this below, computing it with both $\mathbb{Z} / 2 \mathbb{Z}$ and $\mathbb{Z} / 3 \mathbb{Z}$ coefficients

#### **Questions**
* What is the difference with the different coefficients?  Does the diagram look familiar to something we have already seen with $\mathbb{Z} / 2 \mathbb{Z}$ coefficients?

In [ ]:
N = 10000 #Number of initial points in (theta, phi) space
n_sample = 600 #Number of points to evenly subsample
R = 4
r = 2
theta = np.random.rand(N)*2*np.pi
phi = np.random.rand(N)*2*np.pi
X = np.zeros((N, 4))
X[:, 0] = (R + r*np.cos(theta))*np.cos(phi)
X[:, 1] = (R + r*np.cos(theta))*np.sin(phi)
X[:, 2] = r*np.sin(theta)*np.cos(phi/2)
X[:, 3] = r*np.sin(theta)*np.sin(phi/2)

#Now compute persistence diagrams up to H2 with coefficients in Z2 and Z3 of a subsample of the point cloud
# Hint: Rips(coeff=2)  computes homology with coefficients in Z2


#Plot the diagrams and compare


## **Example 5:** A Twisty Curve

For our final example in this notebook, let's take a closed loop that lives on the 3D torus from example 3 which travels twice around the $R$ radius circle for every once it travels around the $r$ radius circle.  We will compute persistent $H_1$ with different metrics on the point cloud.

#### **Questions**
* What is the relationship between the $H_1$ diagram with geodesic and Euclidean distance? Can you explain the birth and death times of the generators?


In [ ]:
NPoints = 400
R = 4
r = 2
theta = np.linspace(0, 2*np.pi, NPoints+1)[0:NPoints]
phi = theta*2
X = np.zeros((NPoints, 3))
X[:, 0] = (R + r*np.cos(theta))*np.cos(phi)
X[:, 1] = (R + r*np.cos(theta))*np.sin(phi)
X[:, 2] = r*np.sin(theta)
xr = [np.min(X.flatten()), np.max(X.flatten())]

#Compute the persistence diagram with the Euclidean distance


#Now, use the following estimator of the geodesic distance to compute the persistence diagram from a distance matrix. 
#The parameter k sets the local neigborhood used in the estimation. A small value of k (such as k=10) should work.

def compute_knn_distance(data, k):
    '''
    Computes the  estimator of geodesic distance of the algorithm ISOMAP.
    '''
    distances = distance_matrix(data,data)
    f_aprox_D = Fermat(1, path_method='D', k=k) 
    f_aprox_D.fit(distances)
    adj_dist = f_aprox_D.get_distances() 
    return adj_dist

d_matrix = compute_knn_distance(X, 10)


#Plot the curve and the persistent diagrams and compare

